In [10]:
import cv2

img = cv2.imread("../input/skin-lesion-analysis-toward-melanoma-detection/skin-lesions/test/melanoma/ISIC_0012258.jpg")

img.shape

In [11]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.show()

In [12]:
train_dir = "../input/skin-lesion-analysis-toward-melanoma-detection/skin-lesions/train"
test_dir = "../input/skin-lesion-analysis-toward-melanoma-detection/skin-lesions/test"
val_dir = "../input/skin-lesion-analysis-toward-melanoma-detection/skin-lesions/valid"

In [13]:
# processing the datasets
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [14]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )
test_data = train_gen.flow_from_directory(test_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )
val_data = train_gen.flow_from_directory(val_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )

In [15]:
train_data.num_classes

In [16]:
label_names = train_data.class_indices.values()

label_names

In [17]:
image, label = train_data.next()

print(image.shape)
print(label.shape)


In [18]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize= (12,12))

for i in range(9):
    image,label = train_data.next()
    plt.subplot(3,3, i+1)
    plt.imshow(image[i])
    plt.title(label[i])
    plt.axis("off")
    

In [19]:
# building a resnet model 
import tensorflow as tf

resnet_base = tf.keras.applications.ResNet50V2(include_top= False)

resnet_base.trainable = False


In [20]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = resnet_base(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation = "relu")(x)
# x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)

resnet_model = tf.keras.Model(inputs,outputs)

In [21]:
# compiling the model

resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"]
)

In [22]:
# fitting data to the model

resnet_model.fit(train_data,
                 epochs = 5,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data)
                 )

## **Tunning the Model**

In [23]:
resnet_base.trainable = True

for layer in resnet_base.layers[:-5]:
  resnet_base.trainable = False

resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)


In [24]:
# fitting data to the tuned model

resnet_model.fit(train_data,
                 epochs = 5,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data)
                 )

In [25]:
resnet_model_evaluation = resnet_model.evaluate(test_data)

In [26]:
print(f"Resnet Model Accuarcy : {resnet_model_evaluation[1] *100 : 0.2f}%")

In [27]:
# visualizing the prediting image

import matplotlib.pyplot as plt
import tensorflow as tf


plt.figure(figsize=(12,12))

for i in range(9):
  image, label = test_data.next()

  resnet_model_prediction = resnet_model.predict(image)
  
  plt.subplot(3,3,i+1)

  plt.imshow(image[i])
  plt.title(f"Prediction : {tf.round(resnet_model_prediction[i])}\n Original : {label[i]}")
  plt.axis("off")

**Train data with Augmentation**

In [28]:
# augmenting train data
import tensorflow as tf

train_aug = tf.keras.preprocessing.image.ImageDataGenerator(
                                             rescale=1/255.,
                                             rotation_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True
                                             )



train_data_aug = train_aug.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=64,
                                               class_mode='categorical')

In [29]:
# fitting data to the tuned model

resnet_model.fit(train_data_aug,
                 epochs = 5,
                 steps_per_epoch = len(train_data_aug),
                 validation_data = val_data,
                 validation_steps = len(val_data)
                 )

In [30]:
# visualizing the prediting data augmented model

import matplotlib.pyplot as plt
import tensorflow as tf


plt.figure(figsize=(12,12))
print("predicting the data augmented model")
for i in range(9):
  image, label = test_data.next()

  resnet_model_prediction = resnet_model.predict(image)
  
  plt.subplot(3,3,i+1)

  plt.imshow(image[i])
  plt.title(f"Prediction : {tf.round(resnet_model_prediction[i])}\n Original : {label[i]}")
  plt.axis("off")

## **Building a EfficientNetB7 model**

In [31]:
# processing the data for efficientnet
import tensorflow as tf
train_data_eff = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode = "categorical",
                                                                     image_size = (224,224),
                                                                     batch_size = 64
                                                                     )
test_data_eff = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode =  "categorical",
                                                                    image_size = (224,224),
                                                                    batch_size = 64
                                                                    )
val_data_eff = tf.keras.preprocessing.image_dataset_from_directory(val_dir,
                                                                   label_mode = "categorical",
                                                                   image_size = (224,224),
                                                                   batch_size = 64
                                                                   )

In [32]:
# building efficintnet model

import tensorflow as tf

eff_base = tf.keras.applications.EfficientNetB7(include_top= False)

eff_base.trainable = False


In [33]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = eff_base(inputs)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(3, activation = "softmax")(x)

eff_model = tf.keras.Model(inputs,outputs)


In [34]:
eff_model.summary()

In [35]:
# compiling the model

eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]

)

In [36]:
# fitting data to the model

eff_model.fit(
    train_data_eff,
    epochs = 5,
    steps_per_epoch = len(train_data_eff),
    validation_data = val_data_eff,
    validation_steps = len(val_data_eff)
)

In [38]:
# tunning the eff model

eff_base.trainable = True

for layer in eff_base.layers[:-5]:
  eff_base.trainable = False

eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

eff_model.fit(train_data_eff,
              epochs = 5,
              steps_per_epoch = len(train_data_eff),
              validation_data = val_data_eff,
              validation_steps = len(val_data_eff)
              )

In [39]:
# model evaluation 

eff_model_evaluation = eff_model.evaluate(test_data_eff)

In [40]:
print(f"Efficient Model Accuarcy: {eff_model_evaluation[1] *100 : 0.2f} %")

In [41]:
eff_labels = test_data_eff.class_names

In [44]:
# visualizing and predicting the model
import matplotlib.pyplot as plt

plt.figure(figsize=(12,12))

for image, label in test_data_eff.take(1):
  eff_model_prediction = eff_model.predict(image)

  for i in range(9):

    plt.subplot(3,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(f"Prediction : {eff_labels[tf.argmax(tf.round(eff_model_prediction[i]))]} \n Oroginal : {eff_labels[tf.argmax(label[i])]}")




